MDST College Football Team 3 Work

By Saad, Bumsik, Zachery

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read game data
games_stats = pd.read_csv('https://raw.githubusercontent.com/skunichetty/cfb-f21/main/data/game_stats.csv')
games = pd.read_csv('https://raw.githubusercontent.com/skunichetty/cfb-f21/main/data/games.csv')

In [3]:
# Merge tables
df = games.merge(games_stats, left_on = 'id', right_on ='id', validate = 'one_to_one')

# Set initial columns
initialColumns = df.columns

In [4]:
# Remove games from our data not from Big Ten matches
df = df[df.conference_game == True]

In [5]:
# Drop features that we don't want to keep
features = ['id', 'away.kickReturnYards','home.kickReturnYards','away.kickReturnTDs',
            'home.kickReturnTDs','away.kickReturns','home.kickReturns','away.interceptionYards',
            'home.interceptionYards','away.passesIntercepted','home.passesIntercepted',
            'away.passesDeflected','home.passesDeflected','away.fumblesLost',
            'home.fumblesLost','away.fumblesRecovered','home.fumblesRecovered',
            'away.totalFumbles','home.totalFumbles','away.defensiveTDs','home.defensiveTDs',
            'away.tacklesForLoss','home.tacklesForLoss','away.tackles','home.tackles',
            'away.qbHurries','home.qbHurries','away.netPassingYards','home.netPassingYards',
            'away.completionAttempts','home.completionAttempts','away.yardsPerPass',
            'home.yardsPerPass','away.rushingAttempts','home.rushingAttempts',
            'away.totalPenaltiesYards','home.totalPenaltiesYards','away.interceptions',
            'home.interceptions','start_time_tbd','neutral_site','venue_id','venue',
            'home_conference','away_conference','highlights','notes','home_line_scores',
            'away_line_scores','week','home_post_win_prob','away_post_win_prob','attendance',
            'start_date','season_type','away_id','home_id','conference_game',
            'home.thirdDownEff','home.fourthDownEff','away.thirdDownEff','away.fourthDownEff']

df.drop(features, axis = 1, inplace = True)

In [6]:
def timeToDec(row):
    row = str(row)
    if len(row) < 4:
      return 0
    if len(row) == 5:
      return float(row[0:2]) + (float(row[3:5]) / 60)
    return float(row[0:1]) + (float(row[2:4]) / 60)

def labeling(row):
    if row > 0:
      return 1
    return 0

def ratioToPercentage(row):
  #print(row)
  if not isinstance(row,str):
    return np.nan
  nums = row.split('-')
  if float(nums[1]) == 0: # [0,0]
    return np.nan

  
  return float(nums[0]) / float(nums[1]) 

In [7]:
df['home.totalTDs'] = df['home.rushingTDs'] + df['home.passingTDs'] + df['home.interceptionTDs'] + df['home.puntReturnTDs']
df['away.totalTDs'] = df['away.rushingTDs'] + df['away.puntReturnTDs'] + df['away.passingTDs'] + df['away.interceptionTDs']
df['point_differential'] = df['home_points'] - df['away_points']
df['turnover_differential'] = df['home.turnovers'] - df['away.turnovers']
df['firstdown_differential'] = df['home.firstDowns'] - df['away.firstDowns']
df['sack_differential'] = df['home.sacks'] - df['away.sacks']

df["home.possessionTime"] = df["home.possessionTime"].apply(timeToDec)
df["away.possessionTime"] = df["away.possessionTime"].apply(timeToDec)



# df['home.thirdDownEff'] = df['home.thirdDownEff'].apply(ratioToPercentage)
# df['away.thirdDownEff'] = df['away.thirdDownEff'].apply(ratioToPercentage)
# df['home.fourthDownEff'] = df['home.fourthDownEff'].apply(ratioToPercentage)
# df['away.fourthDownEff'] = df['away.fourthDownEff'].apply(ratioToPercentage)
df['label'] = df['home_points'] - df['away_points']
df['label'] = df['label'].apply(labeling)

df.fillna(df.mean(),inplace=True)

In [8]:
df

,season,home_team,home_points,home_pregame_elo,home_postgame_elo,away_team,away_points,away_pregame_elo,away_postgame_elo,excitement_index,home.rushingTDs,home.puntReturnYards,home.puntReturnTDs,home.puntReturns,home.passingTDs,home.kickingPoints,home.interceptionTDs,home.firstDowns,home.totalYards,home.rushingYards,home.yardsPerRushAttempt,home.turnovers,home.possessionTime,away.rushingTDs,away.puntReturnYards,away.puntReturnTDs,away.puntReturns,away.passingTDs,away.kickingPoints,away.firstDowns,away.totalYards,away.rushingYards,away.yardsPerRushAttempt,away.turnovers,away.possessionTime,away.interceptionTDs,home.sacks,away.sacks,home.totalTDs,away.totalTDs,point_differential,turnover_differential,firstdown_differential,sack_differential,label
13,2013,Vanderbilt,35.0,1655.0,1644.0,Ole Miss,39.0,1592.0,1603.0,7.582824,3.0,2.000000,0.000000,1.000000,2.0,5.0,0.149799,19.0,426.0,126.0,3.8,2.0,30.516667,5.0,11.000000,0.000000,1.000000,0.0,7.0,28.0,489.0,206.0,5.6,0.0,29.166667,0.000000,2.358484,2.139552,3.954521,5.00000,-4.0,2.0,-9.0,0.218387,0
74,2013,Pittsburgh,13.0,1578.0,1530.0,Florida State,41.0,1818.0,1866.0,2.979730,0.0,15.974504,0.043122,2.028014,1.0,7.0,0.149799,16.0,297.0,96.0,3.6,2.0,24.866667,1.0,26.000000,0.000000,1.000000,4.0,11.0,24.0,533.0,156.0,4.6,0.0,35.133333,0.000000,2.358484,2.139552,3.954521,5.00000,-28.0,2.0,-8.0,0.218387,0
75,2013,East Carolina,31.0,1448.0,1469.0,Florida Atlantic,13.0,1240.0,1219.0,2.797439,2.0,15.974504,0.043122,2.028014,2.0,7.0,0.000000,16.0,312.0,110.0,3.2,2.0,25.950000,0.0,34.000000,0.000000,4.000000,1.0,7.0,16.0,282.0,104.0,3.1,3.0,34.050000,0.140684,2.358484,2.139552,3.954521,3.75381,18.0,-1.0,0.0,0.218387,1
78,2013,Boston College,24.0,1383.0,1404.0,Wake Forest,10.0,1293.0,1272.0,2.840350,1.0,6.000000,0.000000,1.000000,2.0,6.0,0.000000,17.0,314.0,191.0,4.2,1.0,27.916667,0.0,32.000000,0.000000,5.000000,1.0,4.0,16.0,245.0,54.0,1.4,3.0,32.083333,0.000000,2.358484,2.139552,3.000000,1.00000,14.0,-2.0,1.0,0.218387,1
79,2013,Temple,13.0,1426.0,1407.0,Houston,22.0,1421.0,1440.0,8.461062,2.0,2.000000,0.000000,1.000000,0.0,1.0,0.149799,19.0,300.0,100.0,2.6,3.0,29.183333,1.0,6.000000,0.000000,2.000000,0.0,16.0,32.0,524.0,219.0,4.6,2.0,30.116667,0.000000,2.358484,2.139552,3.954521,1.00000,-9.0,1.0,-13.0,0.218387,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6899,2021,Oregon State,42.0,1475.0,1497.0,Utah,34.0,1788.0,1766.0,6.392903,3.0,15.974504,0.043122,2.028014,2.0,6.0,0.149799,22.0,471.0,263.0,6.9,1.0,24.266667,2.0,14.629176,0.045981,1.919285,2.0,10.0,28.0,455.0,188.0,4.6,0.0,33.850000,0.140684,0.000000,2.000000,3.954521,3.75381,8.0,1.0,-6.0,-2.000000,1
6900,2021,TCU,17.0,1662.0,1624.0,West Virginia,29.0,1489.0,1527.0,5.094114,0.0,15.974504,0.043122,2.028014,1.0,5.0,0.149799,19.0,393.0,149.0,4.4,3.0,24.500000,3.0,-2.000000,0.000000,1.000000,0.0,11.0,24.0,487.0,229.0,5.6,0.0,35.500000,0.000000,1.000000,2.000000,3.954521,3.00000,-12.0,3.0,-5.0,-1.000000,0
6901,2021,Indiana,7.0,1531.0,1443.0,Ohio State,54.0,2300.0,2388.0,0.908796,0.0,0.000000,0.000000,1.000000,1.0,1.0,0.149799,9.0,114.0,34.0,0.9,0.0,27.983333,3.0,20.000000,0.000000,1.000000,4.0,10.0,31.0,539.0,187.0,5.8,0.0,32.016667,0.140684,1.000000,5.000000,3.954521,3.75381,-47.0,0.0,-22.0,-4.000000,0
6902,2021,Miami,31.0,1542.0,1546.0,NC State,30.0,1633.0,1629.0,9.472541,0.0,12.000000,0.000000,2.000000,4.0,7.0,0.149799,18.0,422.0,97.0,2.6,0.0,30.833333,1.0,3.000000,0.000000,1.000000,2.0,12.0,18.0,421.0,111.0,5.0,0.0,28.500000,0.140684,1.000000,4.000000,3.954521,3.75381,1.0,0.0,0.0,-3.000000,1


In [9]:
train_seasons = [*range(2013, 2018)]
test_seasons = [*range(2018, 2022)]

print(train_seasons)

train_df = df[df.season.isin(train_seasons)]
test_df = df[df.season.isin(test_seasons)]

[2013, 2014, 2015, 2016, 2017]


In [10]:
train_df.drop(['season'], axis = 1, inplace = True)
test_df.drop(['season'], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
def setup(df):
  dfs = []
  X = []
  y = []  
  teams = {}
  for team, df_team in df.groupby('home_team'):
    dfs.append(df_team)

  for data in dfs:
    team_name = data['home_team'].iloc[0]
    label = data['label']
    
    data.drop('label', axis = 1, inplace=True)
    
    data = data.expanding().mean()
  
    X.append(data)
    


    data_ = data.copy()
    data_['label'] = label
    y.append(label)
    teams[team_name] = data_

 

  return X, y, teams

In [12]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.svm import LinearSVC

In [13]:
X_train, y_train ,teams = setup(train_df)
X_train = pd.concat(X_train)
y_train = pd.concat(y_train)

X_test, y_test, teams_test = setup(test_df)
X_test = pd.concat(X_test)
y_test = pd.concat(y_test)

print(X_train.shape)
print(y_train.shape)


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5) # 50% training and 50% test

(2575, 41)
(2575,)


In [14]:
clf = LinearSVC()
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [15]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred)) # (TP + TN) / (TP + TN + FP + FN)
print("Precision:", metrics.precision_score(y_test, y_pred)) # TP / (TP + FP)
 # fn > fp: good at picking up prdicting true value
print("Recall:", metrics.recall_score(y_test, y_pred)) # TP / (TP + FN)
print("ROC_AUC:", metrics.roc_auc_score(y_test, y_pred)) # 
print("F1 Score:", metrics.f1_score(y_test, y_pred))

Accuracy: 0.5668202764976958
Precision: 0.8880866425992779
Recall: 0.2543950361944157
ROC_AUC: 0.6070414712831637
F1 Score: 0.39549839228295813


In [16]:
# Grid Search
from sklearn.model_selection import GridSearchCV

parameters = {'C': [0.001, 0.01, 0.1, 1, 10]}

clf = GridSearchCV(LinearSVC(), parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)

y_pred = clf.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred)) # (TP + TN) / (TP + TN + FP + FN)
print("Precision:", metrics.precision_score(y_test, y_pred)) # TP / (TP + FP)
print("Recall:", metrics.recall_score(y_test, y_pred)) # TP / (TP + FN)
print("ROC_AUC:", metrics.roc_auc_score(y_test, y_pred)) # 
print("F1 Score:", metrics.f1_score(y_test, y_pred))

# {'C': 0.001}
# Accuracy: 0.4873271889400922
# Precision: 1.0
# Recall: 0.0796277145811789
# ROC_AUC: 0.5398138572905895
# F1 Score: 0.14750957854406127

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

{'C': 0.1}
Accuracy: 0.5633640552995391
Precision: 0.5605797101449276
Recall: 1.0
ROC_AUC: 0.5071521456436932
F1 Score: 0.7184249628528976


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [17]:
def michiganVS(teams, teams_test, opponenet="Ohio State"):
  X = []
  y = []
  label_train = teams['Michigan']['label'] 
  label_test = teams_test['Michigan']['label']
  
  y.append(label_train)
  y.append(label_test) 

  label_train = teams[opponenet]['label'] 
  label_test = teams_test[opponenet]['label']
  y.append(label_train)
  y.append(label_test)

  teams['Michigan'].drop('label', axis = 1, inplace=True)
  teams_test['Michigan'].drop('label', axis = 1, inplace=True)
  teams[opponenet].drop('label', axis = 1, inplace=True)
  teams_test[opponenet].drop('label', axis = 1, inplace=True)
  
  X.append(teams['Michigan'])
  X.append(teams_test['Michigan'])
  X.append(teams[opponenet])
  X.append(teams_test[opponenet])
  
  X = pd.concat(X)
  y = pd.concat(y)

  return X, y 

In [18]:
X, y = michiganVS(teams, teams_test,  opponenet="Ohio State")
prediction = clf.predict(X.iloc[-1:])
print(prediction[-1])
# Michigan Win!

(1,)
1
